In [43]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,split,udf, struct
spark = (SparkSession.builder
    .master("local[*]")
    .config("spark.driver.cores", 1)
    .appName("GP MotoGP Quatar 2014")
    .getOrCreate() )
sc = spark.sparkContext
print(spark)
print(sc)

<SparkContext master=local[*] appName=GP MotoGP Quatar 2014>


In [44]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
customSchema = StructType([StructField("Id", LongType(), True),
                           StructField("Parent_sys_id", StringType(), True),
                           StructField("Source", StringType(), True),
                           StructField("Mentions", StringType(), True),
                           StructField("Target", StringType(), True),
                           StructField("Name_source", StringType(), True),
                           StructField("Body", StringType(), True),
                           StructField("Pub_date", TimestampType(), True),
                           StructField("URLs", StringType(), True),
                           StructField("Tipe_action", StringType(), True),
                           StructField("Link", StringType(), True),
                           StructField("Has_link", ByteType(), True),
                           StructField("Has_picture", ByteType(), True),
                           StructField("Website", StringType(), True),
                           StructField("Country", StringType(), True),
                           StructField("Activity", LongType(), True),
                           StructField("Followers", LongType(), True),
                           StructField("Following", LongType(), True),
                           StructField("Location", StringType(), True)
                          ])

In [45]:
events = spark.read.csv("data/DATASET-Twitter-23-26-Mar-2014-MotoGP-Qatar.csv",
                        header=True, schema=customSchema, timestampFormat="dd/MM/yyyy HH:mm")

In [46]:
#Ejercicio 1.
#Calcular el número total de retweets por usuario para los 50 usuarios con más mensajes
#en la muestra de tweets analizados. Calcular, para cada uno de estos usuarios la media de
#enlaces (URLs) enviados por mensaje. (2.5 puntos).
#Calculamos el top 50 de usuarios.
usuarios_top_50 = (events
                         .groupBy("Source")
                         .agg(count("Id").alias("tweets"))
                         .orderBy("tweets", ascending=False)
                         .limit(50))

#asignamos un alias a este dataframe para luego poder selecionar las columnas 
#cuando hagamos un join.
usuarios_top_50 = usuarios_top_50.alias(alias='usuarios_top_50')

#calculamos el número de rt para el top 50 de usuarios.
numero_de_rt_top_50 = (events
                         .filter(condition=events.Body.contains('rt'))
                         .groupBy("Source")
                         .agg(count("Id").alias("retweets"))
                         .orderBy("retweets", ascending=False))

#asignamos un alias a este dataframe para luego poder selecionar las columnas 
#cuando hagamos un join.
numero_de_rt_top_50 = numero_de_rt_top_50.alias(alias='numero_de_rt_top_50')


#hacemos un left join del top_50 de usuarios con el número de retweets del top 50.
rt_results = usuarios_top_50.join(numero_de_rt_top_50,
                                  col("usuarios_top_50.Source") == col("numero_de_rt_top_50.Source"),
                                  how='left') 
#selecionamos las columnas que queremos imprimir.
final_results=rt_results.select("usuarios_top_50.Source","tweets","retweets").show(50)


+---------------+------+--------+
|         Source|tweets|retweets|
+---------------+------+--------+
|         MotoGP|    95|      36|
|  trackseven707|   152|     152|
|      JesiPacio|   130|     119|
|    VAVEL_motor|   124|      11|
|   MotoGPquotes|   123|     123|
|     m_azharaji|   486|       1|
| RedGhostOnline|   102|      57|
|nisaauliarahma5|   101|     101|
|  keikokoeswadi|   107|       5|
|     kamseunyil|   111|      93|
|     Ciintiia93|   133|     119|
|   Paula_Bravo3|   107|      68|
|       plusmoto|   119|       8|
|     thalia_26_|   119|      70|
|   iNotaMental_|   116|    null|
|       Cev_Ford|   115|      32|
|     MM93Lovers|   169|     169|
|    LiveMoto_GP|   104|      50|
|    AlessiaPont|   182|     181|
|    FansHM_9323|   107|      36|
| AnaAlvarez9325|   111|      64|
|AkbarValeLorenz|    93|    null|
|   qatarflights|   283|     229|
|    MartinoMoto|    96|      33|
|MarcMarquezTeam|   148|      59|
|      johnbokke|   297|     291|
|    yolandaa_

In [47]:
#Calculamos el top 50 de usuarios.
usuarios_top_50 = (events
                         .groupBy("Source")
                         .agg(count("Id").alias("tweets"))
                         .orderBy("tweets", ascending=False)
                         .limit(50))

usuarios_top_50 = usuarios_top_50.alias(alias='usuarios_top_50')


#calculamos el numero de de tweets con enlaces
numero_de_urls_por_tweet = (events
                         .filter(condition=events.Body.contains('http://'))
                         .groupBy("Source")
                         .agg(count("Id").alias("urltweets"))
                         .orderBy("urltweets", ascending=False))

#asignamos un alias a este dataframe para luego poder hacer operaciones con las columnas 
#cuando hagamos un join.
numero_de_urls_por_tweet = numero_de_urls_por_tweet.alias(alias='numero_de_urls_por_tweet')
url_results = usuarios_top_50.join(numero_de_urls_por_tweet,
                                col("usuarios_top_50.Source") == col("numero_de_urls_por_tweet.Source"),
                                 how='left')
#remplazamos los na por ceros para evitar errores a la hora de hacer la división.
final_url_results = url_results.select("usuarios_top_50.Source","urltweets","tweets")
final_results_without_na = final_url_results.na.fill(0)
#con el método with column creamos la columa Avg Url per Tweet que nos da la media de tweets con url 
#entre todos los tweets.
final_results_without_na.withColumn(
    'Avg Url per Tweet', 
    final_results_without_na.urltweets/final_results_without_na.tweets).select("usuarios_top_50.Source",
                                                                               "urltweets","tweets",
                                                                               'Avg Url per Tweet').show(50)


+---------------+---------+------+--------------------+
|         Source|urltweets|tweets|   Avg Url per Tweet|
+---------------+---------+------+--------------------+
|         MotoGP|       38|    95|                 0.4|
|  trackseven707|       81|   152|  0.5328947368421053|
|      JesiPacio|       81|   130|  0.6230769230769231|
|    VAVEL_motor|      110|   124|  0.8870967741935484|
|   MotoGPquotes|       68|   123|  0.5528455284552846|
|     m_azharaji|        1|   486| 0.00205761316872428|
| RedGhostOnline|       34|   102|  0.3333333333333333|
|nisaauliarahma5|       62|   101|  0.6138613861386139|
|  keikokoeswadi|        3|   107|0.028037383177570093|
|     kamseunyil|        5|   111| 0.04504504504504504|
|     Ciintiia93|       68|   133|  0.5112781954887218|
|   Paula_Bravo3|       27|   107|  0.2523364485981308|
|       plusmoto|       19|   119| 0.15966386554621848|
|     thalia_26_|       20|   119| 0.16806722689075632|
|   iNotaMental_|      116|   116|              

In [51]:
#Ejercicio 2.
#Calcular el número total de mensajes que contienen información de geolocalización en
#el campo LOCATION.(2.5 puntos).
#Analizamos una pequeña muestra de los resultados de la columna Location.
events.select('Location').distinct().show(50)
#Observamos como los datos geolocalizados parecen tener el string :UT delante.

+--------------------+
|            Location|
+--------------------+
|  Caracas, Venezuela|
|Brisbane, QLD, Au...|
|Pontianak - Ketapang|
|           claromecó|
|     Dallas/TX, U.S.|
|Vitoria-Gasteiz (...|
|        here and now|
| Orpington, Kent, UK|
|            England |
|Indonesia, Batavi...|
|Gandia països cat...|
|     Jakarta Selatan|
|Temanggung - Yogy...|
|           Bangalore|
|Banjarmasin Indon...|
|        Far Far Away|
|            Ruhrpott|
|             Palermo|
|      Cuenca, España|
| Brisbane, Australia|
|Basque C.Canary I...|
|           adelaide |
|.ÁguilasCapitalDe...|
|            Girardot|
|Santa Coloma, Bar...|
|Concepción/Cañete...|
|                 ...|
|quelque part dans...|
|               Gijon|
|  South East England|
|      Lorenzo's land|
|In a galaxy far, ...|
|    Bekasi - Jakarta|
|   New Delhi, India |
|              Sidrap|
|Italia - Santa Ma...|
|      Ronda (Málaga)|
|           Vila-Real|
|Carabanchel Alto,...|
| Brownsburg, Indiana|
|en el luga

In [49]:
#filtramos los tweets de la columna localizacion con ÜT para confirmar la hipotesis. 
events.filter(condition=events.Location.contains('ÜT:')).select('Location').distinct().show()
#parece pues que los datos geolocalizados contienen las siglas ÜT.


+--------------------+
|            Location|
+--------------------+
|ÜT: 19.7658228,-7...|
|ÜT: -7.782657,110...|
|ÜT: -3.950311,120...|
|ÜT: -6.1743718,10...|
|ÜT: 38.736058,-77...|
|ÜT: -6.234258,106...|
|ÜT: -7.95287,112....|
|ÜT: 1.51355,124.9...|
|ÜT: -6.296696,106...|
|ÜT: -8.175991,111...|
|ÜT: -6.2216544,10...|
|ÜT: 6.1752794,-75...|
|ÜT: 10.469381,-66...|
|ÜT: -6.6852536,10...|
|ÜT: -6.55637,107....|
|ÜT: -7.29323,112....|
|ÜT: 11.428909,-69...|
|ÜT: 3.57574,98.65089|
|ÜT: -6.2082372,10...|
|ÜT: -6.23409,106....|
+--------------------+
only showing top 20 rows



In [52]:
#contamos el número de tweets geolocalizados.
print("Nùmero de tweets con geolocalización {0}".format(
    events.filter(condition=events.Location.contains('ÜT:')).count()))

Nùmero de tweets con geolocalización 2128


In [53]:
#Ejercicio 3.
#Calcular las 10 cuentas de Twitter que más han sido mencionadas en todo el conjunto de
#datos analizados. (2.5 puntos).
#filtramos las menciones del dataset.
mention_events=events.filter(events.Tipe_action == 'MT')
mention_events.select("Mentions").show(10)
#vemos pues como tenemos un string con las posibles menciones.Por lo que dividiremos la columna mentions en 
#varias columnas pues una mención marcmarquez,valeyellow46 no se asignaría de lo contrario a ninguna de las
#dos cuentas.

+--------------------+
|            Mentions|
+--------------------+
|marcmarquez93,val...|
|               aan__|
|       marcmarquez93|
|       marcmarquez93|
|       marcmarquez93|
|rossistas,valeyel...|
|        valeyellow46|
|                null|
|       valeyellow46,|
|       marcmarquez93|
+--------------------+
only showing top 10 rows



In [56]:
#La primera aproximación para realizar este análisis lo haremos sobre la 
#abstracción más básica del módulo de spark RDD.
#Por lo que generamos un RDD a partir de una dataframe.
mentions=events.filter(events.Tipe_action == 'MT').select('Mentions').rdd
#esta función nos divide el dataset por comas y devuelve una array con los
#strings entre medias. 
def split_mentions(text):
    if ',' in str(text):
        #limpiamos el primer y último elemento del texto al convertirlo 
        #a string
        #y devolvemos una array dividio por ','.
        text = (str(text).replace("')","")
                .replace("Row(Mentions='","")
                .split(','))
        return text
    else:
        return text

top_ten_mentions = (mentions.flatMap(lambda line:[(word,1) for word in split_mentions(line)])
                             .reduceByKey(lambda a, b: a + b)
                             .sortBy(lambda x: x[1],ascending=False)
                             .take(10))

for mention in top_ten_mentions:
    print(mention)
#obtenemos los siguientes resultados

('valeyellow46', 19599)
('marcmarquez93', 16826)
('motogp', 7770)
('lorenzo99', 6969)
('26_danipedrosa', 3093)
('alexmarquez23', 1515)
('yamahaindonesia', 1240)
('btsportmotogp', 1107)
('ims', 738)
('movistar_motogp', 570)


In [389]:
#si no quisieramos llegar a la abstracción de rdd propongo esta solución alternativa
#sobre el dataframes.
#hacemos un split por comma de las menciones y creamos una nueva columna que vamos añadiendo a los dataframes.
#para calcular las cuentas con mayor número de menciones, tendremos en cuanta las tres primeras menciones de cada 
#tweet si existen.
first_mention = mention_events.withColumn('Mention',split(mention_events.Mentions, ',')[0])
second_mention = first_mention.withColumn('secondMention',split(mention_events.Mentions, ',')[1])
third_mention = second_mention.withColumn('thirdMention',split(mention_events.Mentions, ',')[2])
#calculamos el numero de de tweets por menciones
first_mention_tweets = (third_mention
                             .groupBy("Mention")
                             .agg(count("Id").alias("firstmentiontweets"))
                             .orderBy("firstmentiontweets", ascending=False))
second_mention_tweets  = (third_mention
                             .where(third_mention.secondMention.isNotNull())
                             .groupBy("secondMention")
                             .agg(count("Id").alias("secondmentiontweets"))
                             .orderBy("secondmentiontweets", ascending=False))

until_second_mention = first_mention_tweets.join(second_mention_tweets, 
                                                 first_mention_tweets.Mention == second_mention_tweets.secondMention,
                                                how='left')
sum_cols = udf(lambda x: x[0]+x[1], IntegerType())
#si tenemos únicamente las dos primeras menciones de cada tweet tendríamos estos resultados
until_second_mention = (until_second_mention.na.fill(0)
                                               .withColumn('Until_Second_Mention', 
                                                           sum_cols(struct('firstmentiontweets', 
                                                                                     'secondmentiontweets')))
                                               .select("Mention","Until_Second_Mention")
                                               .orderBy("Until_Second_Mention", ascending=False)).show(10)


+---------------+--------------------+
|        Mention|Until_Second_Mention|
+---------------+--------------------+
|   valeyellow46|               17880|
|  marcmarquez93|               15952|
|         motogp|                7013|
|      lorenzo99|                6519|
| 26_danipedrosa|                2079|
|  alexmarquez23|                1424|
|  btsportmotogp|                 938|
|yamahaindonesia|                 751|
|movistar_motogp|                 515|
|         rins42|                 437|
+---------------+--------------------+
only showing top 10 rows



In [57]:

#si tuvieras en cuenta hasta la tercera mención.
first_mention = mention_events.withColumn('Mention',split(mention_events.Mentions, ',')[0])
second_mention = first_mention.withColumn('secondMention',split(mention_events.Mentions, ',')[1])
third_mention = second_mention.withColumn('thirdMention',split(mention_events.Mentions, ',')[2])
#calculamos el numero de de tweets por menciones
first_mention_tweets = (third_mention
                             .groupBy("Mention")
                             .agg(count("Id").alias("firstmentiontweets"))
                             .orderBy("firstmentiontweets", ascending=False))
second_mention_tweets  = (third_mention
                             .where(third_mention.secondMention.isNotNull())
                             .groupBy("secondMention")
                             .agg(count("Id").alias("secondmentiontweets"))
                             .orderBy("secondmentiontweets", ascending=False))
third_mention_tweets  = (third_mention
                             .where(third_mention.secondMention.isNotNull())
                             .groupBy("thirdMention")
                             .agg(count("Id").alias("thirdmentiontweets"))
                             .orderBy("thirdmentiontweets", ascending=False))


until_second_mention = first_mention_tweets.join(second_mention_tweets, 
                                                 first_mention_tweets.Mention == second_mention_tweets.secondMention,
                                                how='left')

until_third_mention = until_second_mention.join(third_mention_tweets,
                                                until_second_mention.Mention == third_mention_tweets.thirdMention)
sum_cols = udf(lambda x: x[0]+x[1]+x[2], IntegerType())
#si tenemos únicamente las dos primeras menciones de cada tweet tendríamos estos resultados
until_second_mention = (until_third_mention.na.fill(0)
                                               .withColumn('Until_Third_Mention', 
                                                           sum_cols(struct('firstmentiontweets', 
                                                                           'secondmentiontweets',
                                                                           'thirdmentiontweets')))
                                               .select("Mention","Until_Third_Mention")
                                               .orderBy('Until_Third_Mention', ascending=False)
                                               .limit(10)).show()

#vemos de esta manera que los 7 primeras cuentas continuan siendo las mismas que si tenemos en cuenta hasta dos 
#menciones en cambio si tenemos en cuenta hasta tres menciones la 8,9 y 10 cuenta serían distintas.
#estas cuentas coinciden con la solución propuesta por RDD sin embargo observamos como no es exacta

+---------------+-------------------+
|        Mention|Until_Third_Mention|
+---------------+-------------------+
|   valeyellow46|              19313|
|  marcmarquez93|              16615|
|         motogp|               7535|
|      lorenzo99|               6797|
| 26_danipedrosa|               2694|
|  alexmarquez23|               1495|
|yamahaindonesia|               1224|
|  btsportmotogp|               1046|
|            ims|                640|
|movistar_motogp|                557|
+---------------+-------------------+



In [391]:
# Ejercicio 4.
#Calcular los 10 mensajes más retweeteados y los 10 mensajes que han acumulado más
#respuestas en la muestra de datos analizados. Ahora, restringe la búsqueda a los mensajes
#en el intervalo 2014-03-24 04:00 - 2014-03-24 10:00.
#Mensajes más retweeteados.
retweets_events = events.filter(events.Tipe_action == 'RT')

top_ten_retweets_messages = (retweets_events
                                         .groupBy("Parent_sys_id")
                                         .agg(count("Id").alias("retweets"))
                                         .orderBy("retweets", ascending=False)
                                         .limit(10))

#hacemos un join con el dataset orginial para sacar el cuerpo del mensaje.
parent_messages = events.select("Id","Body")
#unimos el ambos datasets
url_results = top_ten_retweets_messages.join(parent_messages,
                                   top_ten_retweets_messages.Parent_sys_id == parent_messages.Id,
                                   how='inner')
#obtenemos los resultados
url_results.select("Body","retweets").sort(desc("retweets")).show()


+--------------------+--------+
|                Body|retweets|
+--------------------+--------+
|increíble, aún no...|    5515|
|what a race!great...|    4995|
|hoy comienza el m...|    3322|
|bella foto!! @val...|    3236|
|che gara ragazzi!...|    2406|
|another awesome w...|    2266|
|gracias a todos l...|    1774|
|perdonad equipo y...|    1691|
+--------------------+--------+



In [330]:
#Mensajes más contestados.Primero filtramos los mensajes excluyendo retweets.
answer_tweets = events.filter(events.Tipe_action != "RT")
#sacamos por tweet referenciado el mayor número de tweets.
answer_retweets  = (answer_tweets
                                .groupBy("Parent_sys_id")
                                .agg(count("Id").alias("answer_tweets"))
                                .orderBy("answer_tweets", ascending=False)
                                .limit(10))

#hacemos un join con el dataset orginial para sacar el cuerpo del mensaje.
parent_messages = events.select("Id","Body")
#unimos el ambos datasets
url_results = answer_retweets.join(parent_messages,
                                   answer_retweets.Parent_sys_id == parent_messages.Id,
                                   how='inner')
#top 10 retweets.
url_results.select("Body","answer_tweets").sort(desc("answer_tweets")).show()


+--------------------+-------------+
|                Body|answer_tweets|
+--------------------+-------------+
|che gara ragazzi!...|          554|
|perdonad equipo y...|          530|
|increíble, aún no...|          519|
|what a race!great...|          493|
|hoy comienza el m...|          235|
|mirad lo que me a...|          210|
|bella foto!! @val...|          154|
|for those @valeye...|          136|
|circuito di losai...|          126|
+--------------------+-------------+



In [392]:
#Restringiendo el rango de fechas entre 2014-03-24 04:00 - 2014-03-24 10:00 para los retweets.

retweets_events = events.filter(events.Tipe_action == 'RT')
time_retweets_events = retweets_events.where((col("Pub_date") >= "2014-03-24 04:00") & 
                                             (col("Pub_date") <= "2014-03-24 10:00"))

top_ten_retweets_messages = (time_retweets_events
                                         .groupBy("Parent_sys_id")
                                         .agg(count("Id").alias("retweets"))
                                         .orderBy("retweets", ascending=False)
                                         .limit(10))

#hacemos un join con el dataset orginial para sacar el cuerpo del mensaje.
parent_messages = events.select("Id","Body")
#unimos el ambos datasets
url_results = top_ten_retweets_messages.join(parent_messages,
                                   top_ten_retweets_messages.Parent_sys_id == parent_messages.Id,
                                   how='inner')
#top 10 retweets. En el rango de fechas.
url_results.select("Body","retweets").sort(desc("retweets")).show()

+--------------------+--------+
|                Body|retweets|
+--------------------+--------+
|no os recuerda es...|     443|
|increíble, aún no...|     431|
|que foto! valenti...|     368|
|valentino rossi @...|     226|
|perdonad equipo y...|     161|
|@morsellilinda @v...|     140|
|@valeyellow46 fai...|     138|
|que bonito es ver...|      89|
|che gara!! emozio...|      89|
+--------------------+--------+



In [393]:
#Restringiendo el rango de fechas entre 2014-03-24 04:00 - 2014-03-24 10:00 para los answer tweets.
answer_tweets = events.filter(events.Tipe_action != "RT")
time_answer_tweets = answer_tweets.where((col("Pub_date") >= "2014-03-24 04:00") & 
                                             (col("Pub_date") <= "2014-03-24 10:00"))
#sacamos por tweet referenciado el mayor número de tweets.
top_ten_answer_retweets  = (time_answer_tweets
                                .groupBy("Parent_sys_id")
                                .agg(count("Id").alias("answer_tweets"))
                                .orderBy("answer_tweets", ascending=False)
                                .limit(10))

#hacemos un join con el dataset orginial para sacar el cuerpo del mensaje.
parent_messages = events.select("Id","Body")
#unimos el ambos datasets
url_results = top_ten_answer_retweets.join(parent_messages,
                                           top_ten_answer_retweets.Parent_sys_id == parent_messages.Id,
                                           how='inner')
#top 10 retweets.
url_results.select("Body","answer_tweets").sort(desc("answer_tweets")).show()


+--------------------+-------------+
|                Body|answer_tweets|
+--------------------+-------------+
|increíble, aún no...|           64|
|perdonad equipo y...|           46|
|che gara!! emozio...|           12|
|valentino rossi @...|           12|
|no os recuerda es...|            9|
|tres carreras mag...|            9|
|@valeyellow46 fai...|            8|
|the end of a spec...|            7|
|[race report]: @m...|            6|
+--------------------+-------------+

